In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
df = pd.read_csv('labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./Photos/car153.xml,278,503,765,836
1,./Photos/car269.xml,244,402,318,366
2,./Photos/car200.xml,181,316,236,273
3,./Photos/car277.xml,234,392,220,272
4,./Photos/car170.xml,192,393,378,438


In [3]:
import xml.etree.ElementTree as xet

In [4]:
filename = df['filepath'][0]
filename

'./Photos/car153.xml'

In [5]:
def getfilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('./Photos',filename_image)
    return filepath_image

In [6]:
getfilename(filename)

'./Photos/car153.jpg'

In [7]:
image_path = list(df['filepath'].apply(getfilename))
image_path

['./Photos/car153.jpg',
 './Photos/car269.jpg',
 './Photos/car200.jpg',
 './Photos/car277.jpg',
 './Photos/car170.jpg',
 './Photos/car109.jpg',
 './Photos/car69.jpg',
 './Photos/car86.jpg',
 './Photos/car49.jpg',
 './Photos/car244.jpg',
 './Photos/car180.jpg',
 './Photos/car33.jpg',
 './Photos/car10.jpg',
 './Photos/car62.jpg',
 './Photos/car13.jpg',
 './Photos/car142.jpeg',
 './Photos/car124.jpeg',
 './Photos/car251.jpg',
 './Photos/car233.jpg',
 './Photos/car108.jpg',
 './Photos/car155.jpg',
 './Photos/car264.jpg',
 './Photos/car94.jpeg',
 './Photos/car88.jpeg',
 './Photos/car136.jpg',
 './Photos/car28.jpg',
 './Photos/car26.jpg',
 './Photos/car14.jpg',
 './Photos/car253.jpg',
 './Photos/car163.jpg',
 './Photos/car216.jpg',
 './Photos/car246.jpeg',
 './Photos/car140.jpeg',
 './Photos/car256.jpg',
 './Photos/car126.jpeg',
 './Photos/car234.jpg',
 './Photos/car222.jpeg',
 './Photos/car197.jpg',
 './Photos/car266.jpg',
 './Photos/car238.jpg',
 './Photos/car213.jpeg',
 './Photos/car65.jp

#### Verify Image and Output

In [8]:
file_path = image_path[0]
file_path

'./Photos/car153.jpg'

In [9]:
#img = cv2.imread(file_path)

#cv2.namedWindow('example',cv2.WINDOW_NORMAL)
#cv2.imshow('example',img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [12]:
##246xm	478xx	714ym	770yx
img = cv2.imread(file_path)
cv2.rectangle(img,(278,765),(503,836),(0,0,255),3)
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x3ff9e00) is not the object's thread (0x3e4dbb0).
Cannot move to target thread (0x3ff9e00)

QObject::moveToThread: Current thread (0x3ff9e00) is not the object's thread (0x3e4dbb0).
Cannot move to target thread (0x3ff9e00)

QObject::moveToThread: Current thread (0x3ff9e00) is not the object's thread (0x3e4dbb0).
Cannot move to target thread (0x3ff9e00)

QObject::moveToThread: Current thread (0x3ff9e00) is not the object's thread (0x3e4dbb0).
Cannot move to target thread (0x3ff9e00)

QObject::moveToThread: Current thread (0x3ff9e00) is not the object's thread (0x3e4dbb0).
Cannot move to target thread (0x3ff9e00)

QObject::moveToThread: Current thread (0x3ff9e00) is not the object's thread (0x3e4dbb0).
Cannot move to target thread (0x3ff9e00)

QObject::moveToThread: Current thread (0x3ff9e00) is not the object's thread (0x3e4dbb0).
Cannot move to target thread (0x3ff9e00)

QObject::moveToThread: Current thread (0x3ff9e00) is not the object's thread

#### Data Preprocessing

In [13]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

2024-03-31 22:57:37.918766: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-31 22:57:38.967309: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [14]:
labels = df.iloc[:,1:].values

In [15]:
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    #preprocessing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 #normalization
    # normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) #normalized output
    #--------------- append--------------#
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [16]:
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [17]:
X.shape,y.shape

((277, 224, 224, 3), (277, 4))

In [18]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((221, 224, 224, 3), (56, 224, 224, 3), (221, 4), (56, 4))

#### Deep Learning Model ####

In [19]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [21]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False,input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=False

headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)

model = Model(inputs=inception_resnet.input,outputs=headmodel)

2024-03-31 22:59:13.340451: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 76800000 exceeds 10% of free system memory.
2024-03-31 22:59:13.367141: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 76800000 exceeds 10% of free system memory.


In [23]:
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_203 (Conv2D) │ (None, 111, 111,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 111, 111,  │         96 │ conv2d_203[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_203      │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_204 (Conv2D) │ (None, 109, 109,  │      9,216 │ activation_203[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_204[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_204      │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_205 (Conv2D) │ (None, 109, 109,  │     18,432 │ activation_204[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_205[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_205      │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 54, 54,    │          0 │ activation_205[0… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_206 (Conv2D) │ (None, 54, 54,    │      5,120 │ max_pooling2d_4[… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_206[0][0]  │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_206      │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_207 (Conv2D) │ (None, 52, 52,    │    138,240 │ activation_206[0… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_207[0][0]  │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_207      │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 73,663,490 (281.00 MB)

 Trainable params: 19,326,754 (73.73 MB)

 Non-trainable params: 54,336,736 (207.28 MB)

#### Model Training

In [24]:
from tensorflow.keras.callbacks import TensorBoard

In [25]:
tfb = TensorBoard('object_detection')

In [26]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=100,validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - loss: 0.1026 - val_loss: 0.1159
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - loss: 0.1006 - val_loss: 0.0968
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 0.0962 - val_loss: 0.0931
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 0.0863 - val_loss: 0.0856
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 0.0779 - val_loss: 0.0929
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 0.0807 - val_loss: 0.0852
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 0.0783 - val_loss: 0.0837
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 0.0743 - val_loss: 0.0892
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 0.0757 - val_loss: 0.0851
Epoch 10/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 0.0725 - val_loss: 0.0830
Epoch 11/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 0.0757 - val_loss: 0.0841
Epoch 12/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 0.0

In [27]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=200,validation_data=(x_test,y_test),callbacks=[tfb],initial_epoch=101)

Epoch 102/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 2.6240e-04 - val_loss: 0.0059
Epoch 103/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 2.9668e-04 - val_loss: 0.0064
Epoch 104/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 3.1457e-04 - val_loss: 0.0061
Epoch 105/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 4.1006e-04 - val_loss: 0.0065
Epoch 106/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 5.3599e-04 - val_loss: 0.0062
Epoch 107/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 4.3371e-04 - val_loss: 0.0063
Epoch 108/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 3.3144e-04 - val_loss: 0.0059
Epoch 109/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 2.1307e-04 - val_loss: 0.0065
Epoch 110/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 3.6356e-04 - val_loss: 0.0060
Epoch 111/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 2.2912e-04 - val_loss: 0.0058
Epoch 112/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - loss: 1.8727e-04 - val_loss: 0.0062

In [29]:
model.save('./models/object_detection.h5')